<a href="https://colab.research.google.com/github/klien0257/MSIS4013-22520753-DBMS-Project/blob/main/Clip_Semantic_Book_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Collection of methods used to pull down openlibrary data dump and get cover image embeddings using OpenAI's Clip model


In [2]:
import pandas as pd
import requests
import os
import gzip
import json
import numpy as np

import pprint
import tempfile

from typing import Dict, Text

from ast import literal_eval

# import faiss
import torch
import skimage
# import pinecone
import numpy as np
import pandas as pd
from PIL import Image
from io import BytesIO
import IPython.display
import matplotlib.pyplot as plt
# from datasets import load_dataset
from collections import OrderedDict
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer

In [3]:
import PIL

PIL.Image.MAX_IMAGE_PIXELS = None

[link text](https://)Step 1: Download data dump from https://openlibrary.org/data; process in batches with pandas

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [93]:
def download_data_dump():
  chunk_size = 1000000  # Adjust the chunk size as needed
  file_path = '/content/drive/MyDrive/book_data/ol_dump_editions_latest.txt.gz'  # Replace with the path to your large file


  # Read and process each chunk, then output to separate files
  for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunk_size, compression='gzip', header=0, sep='\t', quotechar='"')):
      output_file_path = f'/content/drive/MyDrive/book_data/processed_batches/processed_chunk_{i + 1}.csv'
      chunk.to_csv(output_file_path, index=False)

      print(f"Processed chunk {i + 1}. Output saved to {output_file_path}")

In [94]:
download_data_dump()

Processed chunk 1. Output saved to /content/drive/MyDrive/book_data/processed_batches/processed_chunk_1.csv
Processed chunk 2. Output saved to /content/drive/MyDrive/book_data/processed_batches/processed_chunk_2.csv
Processed chunk 3. Output saved to /content/drive/MyDrive/book_data/processed_batches/processed_chunk_3.csv
Processed chunk 4. Output saved to /content/drive/MyDrive/book_data/processed_batches/processed_chunk_4.csv
Processed chunk 5. Output saved to /content/drive/MyDrive/book_data/processed_batches/processed_chunk_5.csv
Processed chunk 6. Output saved to /content/drive/MyDrive/book_data/processed_batches/processed_chunk_6.csv
Processed chunk 7. Output saved to /content/drive/MyDrive/book_data/processed_batches/processed_chunk_7.csv
Processed chunk 8. Output saved to /content/drive/MyDrive/book_data/processed_batches/processed_chunk_8.csv
Processed chunk 9. Output saved to /content/drive/MyDrive/book_data/processed_batches/processed_chunk_9.csv
Processed chunk 10. Output s

In [5]:

def process_raw_files():
  # Directory where processed chunk files are stored
  processed_chunks_dir = '/content/drive/MyDrive/book_data/processed_batches/'  # Replace with the path to your directory
  select_columns = ['title', 'isbn_10', 'isbn_13', 'publish_date', 'key', 'subjects', 'languages', 'description.value', 'genres']

  # List to store DataFrames of processed chunks
  processed_chunks = []

  # Column names for your DataFrame
  column_names = ['col1', 'col2', 'col3', 'col4', 'col5']  # original data tab separated columns

  # Read each processed chunk file and append to the list
  for i, filename in enumerate(os.listdir(processed_chunks_dir)):
      if filename.endswith('.csv'):
          print(f'Processing file {filename}....')
          file_path = os.path.join(processed_chunks_dir, filename)
          chunk_df = pd.read_csv(file_path, names=column_names)
          books_info_lst = chunk_df['col5'].tolist()

          # raw data stored in column as json
          books_info_lst = [json.loads(book_record) for book_record in books_info_lst]
          books_df = pd.json_normalize(books_info_lst)

          # narrow down amount of data by only outputting english language books and books greater than 100 pages
          books_df['languages_unpacked'] = books_df['languages'].apply(lambda lst: ', '.join(d['key'] for d in lst) if isinstance(lst, list) else '')
          df_languages_keys = books_df[books_df['languages_unpacked'] =='/languages/eng']
          df_page_cnt_filter = df_languages_keys[df_languages_keys['number_of_pages'] >= 100]

          final_df = df_page_cnt_filter
          final_df = final_df[select_columns]
          final_df.to_csv(f'/content/drive/MyDrive/book_data/all_raw/books_{i}.csv')



In [7]:
process_raw_files()

Processing file processed_chunk_1.csv....
Processing file processed_chunk_2.csv....
Processing file processed_chunk_3.csv....
Processing file processed_chunk_4.csv....
Processing file processed_chunk_5.csv....
Processing file processed_chunk_6.csv....
Processing file processed_chunk_7.csv....
Processing file processed_chunk_8.csv....
Processing file processed_chunk_9.csv....
Processing file processed_chunk_10.csv....
Processing file processed_chunk_11.csv....
Processing file processed_chunk_12.csv....
Processing file processed_chunk_13.csv....
Processing file processed_chunk_14.csv....
Processing file processed_chunk_15.csv....
Processing file processed_chunk_16.csv....
Processing file processed_chunk_17.csv....
Processing file processed_chunk_18.csv....
Processing file processed_chunk_19.csv....
Processing file processed_chunk_20.csv....
Processing file processed_chunk_21.csv....
Processing file processed_chunk_22.csv....
Processing file processed_chunk_23.csv....
Processing file proc

Step 2: Read in full data dump and filter for desired data (in this case recent (between 2020-2023) adult fiction

In [28]:
def read_csvs_in_directory(directory):
    dataframes = []
    for filename in os.listdir(directory):
        print(f'Processing file {filename}....')
        if filename.endswith(".csv"):
            csv_path = os.path.join(directory, filename)
            books_df = pd.read_csv(csv_path)

            dataframes.append(books_df)
    # return dataframes
    return pd.concat(dataframes, ignore_index=True)

In [29]:
read_csvs_in_directory (all_raw)

Processing file books_0.csv....
Processing file books_1.csv....
Processing file books_2.csv....
Processing file books_3.csv....
Processing file books_4.csv....
Processing file books_5.csv....
Processing file books_6.csv....
Processing file books_7.csv....
Processing file books_8.csv....
Processing file books_9.csv....
Processing file books_10.csv....
Processing file books_11.csv....
Processing file books_12.csv....
Processing file books_13.csv....
Processing file books_14.csv....
Processing file books_15.csv....
Processing file books_16.csv....
Processing file books_17.csv....
Processing file books_18.csv....
Processing file books_19.csv....
Processing file books_20.csv....
Processing file books_21.csv....
Processing file books_22.csv....
Processing file books_23.csv....
Processing file books_24.csv....
Processing file books_25.csv....
Processing file books_26.csv....
Processing file books_27.csv....
Processing file books_28.csv....
Processing file books_29.csv....
Processing file book

,Unnamed: 0,title,isbn_10,isbn_13,publish_date,key,subjects,languages,description.value,genres
0,4,Encyclopedia of designs for quilting,['0891458875'],NaN,1996,/books/OL1000121M,['Quilting -- Patterns.'],[{'key': '/languages/eng'}],NaN,NaN
1,9,Peacemaking strategies in Northern Ireland,['0312163460'],NaN,1997,/books/OL1000199M,"['Conflict management -- Northern Ireland.', '...",[{'key': '/languages/eng'}],Competing theories of conflict management can ...,NaN
2,34,Musicology and performance,['0300068050'],NaN,1997,/books/OL1000711M,"['Music -- History and criticism', 'Musicology...",[{'key': '/languages/eng'}],Arriving in the United States at age twenty-se...,NaN
3,40,The return of the wolf to Yellowstone,['0805031014'],NaN,1997,/books/OL1000771M,['Wolves -- Reintroduction -- Yellowstone Nati...,[{'key': '/languages/eng'}],NaN,NaN
4,48,The Mystery of the Secret Message,"['0807554294', '0807554308']",NaN,1996,/books/OL1000890M,"['Brothers and sisters -- Fiction.', 'Orphans ...",[{'key': '/languages/eng'}],While helping their grandfather prepare for Gr...,NaN
...,...,...,...,...,...,...,...,...,...,...
12745832,308028,"Isolation, characterization, and utilization o...",['3540616969'],NaN,1997,/books/OL999641M,"['Developmental neurophysiology.', 'Neural ste...",[{'key': '/languages/eng'}],In trying to understand postnatal neurogenesis...,NaN
12745833,308032,An investigation into the thought of Kōgyō D...,NaN,NaN,1998,/books/OL99967M,NaN,[{'key': '/languages/eng'}],NaN,NaN
12745834,308038,Turn the cup around,['0385322925'],NaN,1997,/books/OL999767M,"['Grandmothers -- Fiction.', 'Brothers and sis...",[{'key': '/languages/eng'}],Mysterious paintings in a cave near her grandm...,NaN
12745835,308040,Abnormal illness behaviour,['0471965731'],NaN,1997,/books/OL999781M,"['Somatoform disorders.', 'Hypochondriasis -- ...",[{'key': '/languages/eng'}],NaN,NaN


In [34]:
def get_image_id(row):
    isbn_10 = row['isbn_10']
    isbn_13 = row['isbn_13']
    image_id = None
    if isbn_10 is not np.nan:
        image_id = literal_eval(isbn_10)[0]
    elif isbn_13 is not np.nan:
        image_id = literal_eval(isbn_13)[0]

    return image_id

In [37]:
def get_image_url(image_id):
  if image_id:
    return f'https://covers.openlibrary.org/b/isbn/{image_id}.jpg'
  return None

In [38]:
def get_filtered_data():
  parent_directory = '/content/drive/MyDrive/book_data/all_raw'
  result_df = read_csvs_in_directory(parent_directory)
  fiction_df = result_df[result_df['subjects'].fillna('').str.contains('Fiction', case=False)]

  adult_fiction_df = fiction_df[~fiction_df['subjects'].str.contains('Juvenile', case=False)]

  recent_fiction_df = adult_fiction_df[adult_fiction_df['publish_date'].fillna('').str.contains('|'.join(['2020', '2021', '2022', '2023']), case=False)]

  # get image id which is isbn_13 or isbn_13 code then use to build urls

  recent_fiction_df['image_id'] = recent_fiction_df.apply(get_image_id, axis=1)
  recent_fiction_df['image_url'] = recent_fiction_df['image_id'].apply(get_image_url)

  recent_fiction_df = recent_fiction_df[recent_fiction_df['image_url'].notna()]

  recent_fiction_df.to_csv('/content/drive/MyDrive/book_data/recent_fiction.csv')

In [43]:
# Execute this cell
get_filtered_data()

Processing file books_0.csv....
Processing file books_1.csv....
Processing file books_2.csv....
Processing file books_3.csv....
Processing file books_4.csv....
Processing file books_5.csv....
Processing file books_6.csv....
Processing file books_7.csv....
Processing file books_8.csv....
Processing file books_9.csv....
Processing file books_10.csv....
Processing file books_11.csv....
Processing file books_12.csv....
Processing file books_13.csv....
Processing file books_14.csv....
Processing file books_15.csv....
Processing file books_16.csv....
Processing file books_17.csv....
Processing file books_18.csv....
Processing file books_19.csv....
Processing file books_20.csv....
Processing file books_21.csv....
Processing file books_22.csv....
Processing file books_23.csv....
Processing file books_24.csv....
Processing file books_25.csv....
Processing file books_26.csv....
Processing file books_27.csv....
Processing file books_28.csv....
Processing file books_29.csv....
Processing file book

<ipython-input-38-863235399>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent_fiction_df['image_id'] = recent_fiction_df.apply(get_image_id, axis=1)
<ipython-input-38-863235399>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recent_fiction_df['image_url'] = recent_fiction_df['image_id'].apply(get_image_url)


Step 3: Batch through filtered dataset, pull down images, and save embeddings

In [45]:
def get_image(image_URL):
  if image_URL:
    try:
      response = requests.get(image_URL)
      image = Image.open(BytesIO(response.content)).convert("RGB")
      width, height = image.size
      if width == 1 and height == 1:
        return None
      return image
    except:
      print(f"Error: {image_URL}")
      return None
    return None

In [46]:
def get_embedding_batch(images):

  model_ID = "openai/clip-vit-base-patch32"
  device = "cuda" if torch.cuda.is_available() else "cpu"

  model = CLIPModel.from_pretrained(model_ID).to(device)
  processor = CLIPProcessor.from_pretrained(model_ID)
      # Get the tokenizer
  tokenizer = CLIPTokenizer.from_pretrained(model_ID)

  try:
    batch_tensor = torch.stack([processor(images=img, return_tensors="pt", padding=True)['pixel_values'][0] for img in images])
    embedding = model.get_image_features(batch_tensor)

    return embedding.cpu().detach().numpy()
  except:
      print(f'Error: image in batch does not exist or cannot be downloaded')
      return []
  return []

In [52]:
def batch_process_get_embeddings():
  filtered_df = pd.read_csv('/content/drive/MyDrive/book_data/recent_fiction.csv')
  directory_path = '/content/drive/MyDrive/book_data/batched_embeddings/'
  directory_files = os.listdir(directory_path)

  # get iterator starting point from file name to kick process off at file where it failed
  latest_ind = 0
  if len(directory_files) > 0:
    latest_file = sorted(directory_files)[len(directory_files) - 1]
    latest_ind = int(latest_file.split('_')[1])

  step_size = 100
  images = []
  selected_rows = None
  for start_row in range(latest_ind, 10000, step_size):
      print(f'Processing from row {start_row}')
      # Calculate the end row for each iteration
      end_row = min(start_row + step_size - 1, len(filtered_df) - 1)

      # Get the group of rows for the current iteration
      selected_rows = filtered_df.iloc[start_row:end_row + 1]


      image_embedding_df_cols = ['key', 'image']


      selected_rows["image"] = selected_rows[str("image_url")].apply(get_image)

      image_df = selected_rows[selected_rows['image'].notna()]
      image_df = image_df[image_embedding_df_cols]


      image_embeddings = []
      chunk_size = 5
      for i in range(0, len(image_df['image'].values), chunk_size):
        print(f'Processing images from {i}')
        chunk_df = image_df.iloc[i:i + chunk_size]

        chunk_embeddings = get_embedding_batch(chunk_df['image'].values)

        image_embeddings.extend(list(chunk_embeddings))



      image_df['image_embeddings'] = list(image_embeddings)


      merged_df = pd.merge(selected_rows, image_df, on='key', how='left')


      merged_df.to_csv(f'/content/drive/MyDrive/book_data/batched_embeddings/{start_row}_{start_row + step_size}_w_embeddings.csv')


In [53]:
# Execute this cell
batch_process_get_embeddings()

Processing from row 0


<ipython-input-52-3870544766>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rows["image"] = selected_rows[str("image_url")].apply(get_image)


Processing images from 0


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

...

<ipython-input-52-3870544766>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rows["image"] = selected_rows[str("image_url")].apply(get_image)


Processing images from 0
Processing images from 5
Processing images from 10
Processing images from 15
Processing images from 20
Processing images from 25
Processing images from 30
Processing images from 35
Processing images from 40
Processing from row 7800


<ipython-input-52-3870544766>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rows["image"] = selected_rows[str("image_url")].apply(get_image)


Processing images from 0
Processing images from 5
Processing images from 10
Processing images from 15
Processing images from 20
Processing images from 25
Processing images from 30


KeyboardInterrupt: 

In [54]:
def pkl_embeddings():
  result_df = read_csvs_in_directory('/content/drive/MyDrive/book_data/batched_embeddings/')
  result_df['image_embeddings'] = result_df['image_embeddings'].str.strip('[]').str.split().apply(lambda x: np.array(x).astype(float)).to_numpy()
  image_df = result_df[result_df['image_embeddings'].notna()]
  image_df['image_embeddings'] = image_df['image_embeddings'].apply(lambda x: x.reshape(1, -1))

  image_df = image_df[image_df['image_embeddings'].notna()]

  image_df.to_pickle('/content/drive/MyDrive/book_data/image_embeddings.pkl');

In [55]:
 pkl_embeddings()

Processing file 0_100_w_embeddings.csv....
Processing file 100_200_w_embeddings.csv....
Processing file 200_300_w_embeddings.csv....
Processing file 300_400_w_embeddings.csv....
Processing file 400_500_w_embeddings.csv....
Processing file 500_600_w_embeddings.csv....
Processing file 600_700_w_embeddings.csv....
Processing file 700_800_w_embeddings.csv....
Processing file 800_900_w_embeddings.csv....
Processing file 900_1000_w_embeddings.csv....
Processing file 1000_1100_w_embeddings.csv....
Processing file 1100_1200_w_embeddings.csv....
Processing file 1200_1300_w_embeddings.csv....
Processing file 1300_1400_w_embeddings.csv....
Processing file 1400_1500_w_embeddings.csv....
Processing file 1500_1600_w_embeddings.csv....
Processing file 1600_1700_w_embeddings.csv....
Processing file 1700_1800_w_embeddings.csv....
Processing file 1800_1900_w_embeddings.csv....
Processing file 1900_2000_w_embeddings.csv....
Processing file 2000_2100_w_embeddings.csv....
Processing file 2100_2200_w_embedd

<ipython-input-54-99353420>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  image_df['image_embeddings'] = image_df['image_embeddings'].apply(lambda x: x.reshape(1, -1))
